In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1607488703100_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession \
    .builder \
    .appName("graph-job") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.10.0") \
    .config("spark.jars.packages", "neo4j-contrib:neo4j-connector-apache-spark_2.11:4.0.0") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.1-spark2.4-s_2.11") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark.sparkContext.addPyFile("/lib/spark/jars/graphframes-0.8.1-spark2.4-s_2.11.jar")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from graphframes import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_fundo = spark.read.format("org.neo4j.spark.DataSource") \
    .option("url", url) \
    .option("authentication.basic.username", username) \
    .option("authentication.basic.password", password) \
    .option("query", """match (f:Fundo) return ID(f) as id, f.cnpj as cnpj,
    f.razao_social as nome, f.classe as classe""") \
    .load()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_fundo.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------+--------------------+------------------+
| id|          cnpj|                nome|            classe|
+---+--------------+--------------------+------------------+
|  0|36017639000127|051 7010 FUNDO DE...|Fundo Multimercado|
|  1|34172417000153|051 AÇÕES FUNDO D...|    Fundo de Ações|
|  2|36016411000112|051 ALLOCATION FI...|Fundo Multimercado|
|  3|36498260000186|051 ALOCAÇÃO AÇÕE...|    Fundo de Ações|
|  4|36498567000187|051 ALOCAÇÃO MODE...|Fundo Multimercado|
|  5|03998582000135|051 BLUE MARLIN F...|Fundo Multimercado|
|  6|33913620000170|051 BRUCUTU FUNDO...|Fundo Multimercado|
|  7|37428669000199|051 CAJUINA ALPHA...|    Fundo de Ações|
|  8|24498658000133|051 CAJUINA FUNDO...|    Fundo de Ações|
|  9|31964019000181|051 CAPITAL - COL...|Fundo Multimercado|
| 10|30509213000104|051 CARIBE FUNDO ...|Fundo Multimercado|
| 11|36248874000100|051 CP ALLOCATION...|Fundo Multimercado|
| 12|35820670000139|051 FUTURA FUNDO ...|Fundo Multimercado|
| 13|33150563000115|051 

In [15]:
df_rel = spark.read.format("org.neo4j.spark.DataSource") \
    .option("url", url) \
    .option("authentication.basic.username", username) \
    .option("authentication.basic.password", password) \
    .option("query", """match p=(:Fundo)-[r:COMPRA]->(:Fundo) where r.weight >= 0.03
    return ID(startNode(r)) as src, ID(endNode(r)) as dst, r.weight as weight""") \
    .load()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_rel.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---+--------------------+
|  src|dst|              weight|
+-----+---+--------------------+
|   28|  1| 0.13378761384238716|
|   12|  1| 0.15332622558254408|
|   26|  1|  0.2927373742531345|
|    2|  1| 0.14440720953492098|
|    6|  1| 0.10513828752573881|
|   23|  1| 0.13938181635551863|
|    9|  1| 0.07560055780846033|
|   13|  1|  0.1339341628806844|
|   10|  1| 0.04055959305931201|
|   18|  1| 0.15358691249320475|
|   20|  1| 0.07681447425098697|
|31764|  1|  0.1922158042668773|
|25011|  1|0.049008986712385176|
|    5|  1| 0.13164392739110897|
|    4|  2|  0.9916931986291865|
|   12|  7| 0.11253203972999327|
|   28|  7| 0.10797267382376545|
|    9|  7| 0.05344233711776862|
|    6|  7| 0.04616400601099706|
|   18|  7| 0.03286421631140249|
+-----+---+--------------------+
only showing top 20 rows

In [17]:
g = GraphFrame(df_fundo, df_rel)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
results = g.pageRank(resetProbability=0.01, maxIter=20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_page = results.vertices \
    .select("id", "cnpj", "nome", "classe", "pagerank") \
    .orderBy("pagerank", ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_page.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------+--------------------+-------------------+------------------+
|   id|          cnpj|                nome|             classe|          pagerank|
+-----+--------------+--------------------+-------------------+------------------+
|36774|07455507000189|VERDE MASTER FUND...| Fundo Multimercado|180.63336725639252|
|34150|12798221000136|SPX NIMITZ MASTER...| Fundo Multimercado| 158.6373613001634|
| 8072|09215250000113|BTG PACTUAL TESOU...|Fundo de Renda Fixa|150.26929647922344|
| 4304|08915927000163|BEM FUNDO DE INVE...|Fundo de Renda Fixa|  144.542907610638|
|24909|29236556000163|LEGACY CAPITAL MA...| Fundo Multimercado|142.24288330036975|
|23509|11419818000160|ITAÚ VERSO A REND...|Fundo de Renda Fixa|116.99989723960562|
|24329|12083748000184|KAPITALO MASTER I...| Fundo Multimercado|116.86561700826277|
|34166|12808980000132|SPX RAPTOR MASTER...| Fundo Multimercado|101.52921798491579|
| 2262|11188572000162|ATMOS MASTER FUND...|     Fundo de Ações| 97.73430467662625|
|  4